<a href="https://colab.research.google.com/github/timeseriesAI/tsai/blob/master/tutorial_nbs/05_TS_archs_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# 设置proxy
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

# Import libraries 📚

In [1]:
from tsai.all import *
from IPython.display import clear_output
my_setup()

os              : Linux-5.15.0-97-generic-x86_64-with-glibc2.17
python          : 3.8.18
tsai            : 0.3.9
fastai          : 2.7.14
fastcore        : 1.5.29
torch           : 2.2.1+cu121
device          : 1 gpu (['NVIDIA GeForce GTX 1060 6GB'])
cpu cores       : 6
threads per cpu : 2
RAM             : 31.22 GB
GPU memory      : [6.0] GB


# Experiments 🧪

我运行了一个小测试来展示如何使用 create_model 函数在 tsai 库中构建任何模型。
> I've run a small test to show how you can build any model in the `tsai` library using the `create_model` functions.

最简单的方法是传递您想要使用的架构、 TSDataLoaders 和您想要使用的任何 kwargs 。 create_model 函数将从 TSDataLoaders 对象中获取必要的数据。

> The esiest way to do it is pass the architecture you want to use, a `TSDataLoaders` and any `kwargs` you'd like to use. The `create_model` function will pick up the necessary data from the `TSDataLoaders` object.

我使用了 2 个多元时间序列数据集。正如您所看到的，很难预测哪种架构最适合哪个数据集。
> I've used 2 multivariate time series datasets. As you can see, it's difficult to predict which architecture will work best for which dataset.

请记住，这是一个非常简单的测试，没有任何超参数调整。
>Please, bear in mind that this is a very simple test without any hyperparameter tuning.

In [3]:
dsid = 'NATOPS' 
bs = 64
X, y, splits = get_UCR_data(dsid, return_split=False)
print(X.shape)
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2])

archs = [(FCN, {}), (ResNet, {}), (xresnet1d34, {}), (ResCNN, {}), 
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}), 
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True}),
         (LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]

results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy', 'time'])
for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    learn = Learner(dls, model,  metrics=accuracy)
    start = time.time()
    learn.fit_one_cycle(100, 1e-3)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]
    results.loc[i] = [arch.__name__, k, count_parameters(model), vals[0], vals[1], vals[2], int(elapsed)]
    results.sort_values(by='accuracy', ascending=False, kind='stable', ignore_index=True, inplace=True)
    clear_output()
    display(results)

,arch,hyperparams,total params,train loss,valid loss,accuracy,time
0,ResNet,{},490758,0.029529,0.101087,0.966667,4
1,FCN,{},285446,0.069985,0.121947,0.955556,4


TypeError: __init__() got an unexpected keyword argument 'seq_len'

In [4]:
dsid = 'LSST' 
bs = 64
X, y, splits = get_UCR_data(dsid, return_split=False)
print(X.shape)
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2])

archs = [(FCN, {}), (ResNet, {}), (xresnet1d34, {}), (ResCNN, {}), 
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}), 
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True}),
         (LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]

results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy', 'time'])
for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    learn = Learner(dls, model,  metrics=accuracy)
    start = time.time()
    learn.fit_one_cycle(100, 1e-3)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]
    results.loc[i] = [arch.__name__, k, count_parameters(model), vals[0], vals[1], vals[2], int(elapsed)]
    results.sort_values(by='accuracy', ascending=False, kind='stable', ignore_index=True, inplace=True)
    clear_output()
    display(results)

,arch,hyperparams,total params,train loss,valid loss,accuracy,time
0,FCN,{},270350,0.837919,1.353478,0.610706,28
1,ResNet,{},482574,0.029235,2.298482,0.595296,50


TypeError: __init__() got an unexpected keyword argument 'seq_len'